In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
from torchvision.models import densenet121
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
#трансформации для изображения
transform = transforms.Compose([
    torchvision.transforms.Resize(
        size=(224, 224)
    ),
    transforms.ToTensor(),
])

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
#загружаем предобученную модель
model = densenet121()
model.load_state_dict(torch.load('DenseNet_79.ckpt'))

<All keys matched successfully>

In [29]:
model = model.to(device)

In [30]:
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [49]:
#загрузка изображения
image = Image.open('casual_826.png')
image = image.convert('RGB')

In [50]:
#применяем трансформации к изображению
image_t = transform(image)
batch_t = torch.unsqueeze(image_t, 0)

In [51]:
#получаем выход модели
out = model(batch_t.to(device))
print(out.shape)

torch.Size([1, 1000])


In [52]:
#получаем вероятности для каждого класса
probabilities = torch.nn.functional.softmax(out[0], dim=0)

In [53]:
#тут выводятся все вероятности принадлежности к классам, но надо брать самую большую, то есть torch.topk(probabilities, 1)  

top3_prob, top3_catid = torch.topk(probabilities, 3)
categories = ['casual', 'sport', 'formal']


for i in range(top3_prob.size(0)):
    print(categories[top3_catid[i]], top3_prob[i].item())

casual 0.9996823072433472
formal 0.00031681000837124884
sport 3.2107880088005913e-07
